In [1]:
import sqlite3
from IPython.display import Markdown
import re

In [2]:
def PrintTable(liste, desc=None):
    TableString = ""
    if desc is not None:
        TableString += "|".join([i[0] for i in desc]) + '\n'
        TableString += "|".join(['-'*len(i[0]) for i in desc]) + '\n'

    for l in liste:
        TableString += "|".join([str(i) for i in l]) + '\n'

    return TableString

In [3]:
# Type your Data in here 

# ItemSID (Type in a internal StringIdentifier. ITEM_ will be added by this script)
# (e.g. FIRE_SWORD or POSION_APPLE)
ItemSID = 'POSION_APPLE'
ItemNameEnglish = 'Posioned Apple'

# please type in a Float
ItemWeight = .1

# ItemType (Type a letter representing the type of the Item)
# (C -> Consumable, A -> Armor, W -> Weapon)
ItemType = 'C'

# optional
ItemNameGerman = ''
ItemNameFrench = ''

ItemDescEnglish = ''
ItemDescGerman = ''
ItemDescFrench = ''

# Item Type specifics
# Consumable

# ItemTarget (The Target the Item can be used on) 
# (C -> only used on Character (e.g. Health Potion), I -> only used on Item (e.g. ), A -> used on All (e.g. Posion)) 
itemTarget = 'C'

# Useable in Fight (Should the item be able to be used in)
itemInFight = False

# Armor / Shield

# ArmorBonus (The bonus for the armor class of the character wearing the armor) 
# int
ArmorBonus = 0
# ArmorType (the Type of armor) 
# (C -> Cloth, L -> Light armor, M -> middle armor, H -> Heavy Armor, S -> Shield)
ArmorType = None
# ArmorPart (The body part the Armor should be able to be worn) 
# (S -> Shoes, L -> Legs, T -> Torso, G -> Gloves, N -> Neckless, H -> Helmet, S -> Shield)
ArmorPart = None

# Weapon

# AttackBonus (The bonus for each attack of the character wielding a Weapon)
# int 
AttackBonus = None

# StandardDamage (The Standard Damage of the Weapon without bonus)
# (nWm where n is the number of dice and m the number of sides)
StandardDamage = '4W4'

# WeaponRange (the Range of the Weapon)
# int
WeaponRange = None
# WeaponClass (the class/type of Weapon (e.g. Sword, Javelin, etc.))
WeaponClass = None

In [5]:
# Check input
if ItemSID is None:
    raise ValueError("ItemSID must be given")

ItemSID = ItemSID.upper()
ItemSID = ItemSID.replace(" ", "_")


if ItemNameEnglish is None:
    raise ValueError("English Name must be given")

if ItemType not in ["C", "A", "W"]:
    raise ValueError("ItemType must be C, A or W")

match ItemType:
    case 'C':
        if itemTarget not in ["C", "I", "A"]:
            raise ValueError(" \"ItemTarget\" must be C, I or A")
        if itemInFight not in ["T", "F", "0", "1", True, False]:
            raise ValueError(" \"Useable in Fight\" must be T or F")
        itemInFight = 0 if itemInFight in ["F", "0", False] else 1
    case 'A':
        if ArmorType not in ["C", "L", "M", "H", "S"]:
            raise ValueError(" \"ItemTarget\" must be C, L, M or H")
        if ArmorPart not in ["S", "L", "T", "G", "N", "H", "S"]:
            raise ValueError(" \"ItemTarget\" must be S, L, T, G, N or H")
    case 'W':
        if not re.match(r"\d{0,2}W\d{1,2}", StandardDamage):
            raise ValueError("StandardDamage must be of shape nWm (where n is the number of dice and m the number of sides)")
        

In [6]:
con = sqlite3.connect("DND6/Databases/Objects.db")

In [7]:
# Save Data to DB
cur = con.cursor()

NameSID = f"ITEMNAME_{ItemSID}"
DescSID = f"ITEMDESC_{ItemSID}" if not ItemDescEnglish == '' else "DESC_EMPTY"

# Store Name
sqlStatementName = f"""INSERT INTO Names (NameSID, English, German, French)
VALUES ("{NameSID}", "{ItemNameEnglish}", "{ItemNameGerman}", "{ItemNameFrench}");"""
cur.execute(sqlStatementName)

if DescSID != 'DESC_EMPTY':
    sqlStatementDescription = f"""INSERT INTO Descriptions (DescSID, English, German, French)
    VALUES ("{DescSID}", "{ItemDescEnglish}", "{ItemDescGerman}", "{ItemDescFrench}");"""

    cur.execute(sqlStatementDescription)

# Store Titem
sqlStatementItemBase = f"""INSERT INTO Item (SID, NameSID, DescSID, Weight, Type)
VALUES ("ITEM_{ItemSID}", "{NameSID}", "{DescSID}", {ItemWeight}, "{ItemType}");"""

cur.execute(sqlStatementItemBase)

match ItemType:
    case 'C':
        sqlStatementItemChild = f"""INSERT INTO ItemConsumable (ItemSID, UseTarget, UseableInFight)
        VALUES ("ITEM_{ItemSID}", "{itemTarget}", "{itemInFight}");"""
    case 'A':
        sqlStatementItemChild = f"""INSERT INTO ItemArmor (ItemSID, ArmorBonus, ArmorType, ArmorPart)
        VALUES ("ITEM_{ItemSID}", {ArmorBonus}, "{ArmorType}", "{ArmorPart}");"""
    case 'S':
        sqlStatementItemChild = f"""INSERT INTO ItemShield (ItemSID, ArmorBonus)
        VALUES ("ITEM_{ItemSID}", {ArmorBonus});"""
    case 'W':
        sqlStatementItemChild = f"""INSERT INTO ItemWeapon (ItemSID, AttackBonus, StandardDamage, Range, Class)
        VALUES ("ITEM_{ItemSID}", {AttackBonus}, "{StandardDamage}", {WeaponRange}, "{WeaponClass}");"""

cur.execute(sqlStatementItemChild)

In [1]:
match ItemType:
    case 'C':
        sqlStatementgetItem = f"""  SELECT SID, Names.English NameEnglish, Names.German NameGerman, Names.French NameFrench, 
									Descriptions.English DescEnglish, Descriptions.German DescGerman, Descriptions.French DescFrench, 
									Item.Weight, Item.InventoryIcon
									FROM Item
									JOIN Names ON Item.NameSID = Names.NameSID
									JOIN Descriptions ON Item.DescSID = Descriptions.DescSID
									JOIN ItemConsumable ON Item.SID = ItemConsumable.ItemSID
									WHERE Item.Type='C';"""
    case 'A':
        pass
    case 'S':
        pass
    case 'W':
        pass

cur = cur.execute()
Markdown(PrintTable(cur.fetchall(), cur.description))

NameError: name 'ItemType' is not defined

In [ ]:
# con.commit()
con.close()